In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression, LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
import csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:

train = pd.read_csv("/kaggle/input/bigquery-geotab-intersection-congestion/train.csv")
test = pd.read_csv("/kaggle/input/bigquery-geotab-intersection-congestion/test.csv")

In [ ]:
train.head()

# TODO

~~1. categorize address~~

~~2. if entry address is the same as exit address~~

3. convert directions





In [ ]:
address_encoding = {
    "Street": 0,
     "St": 0,
     "Avenue": 1,
     "Ave": 1,
     "Boulevard": 2,
     "Road": 3,
     "Drive": 4,
     "Lane": 5,
     "Tunnel": 6,
     "Highway": 7,
     "Way": 8,
     "Parkway": 9,
     "Parking": 10,
     "Oval": 11,
     "Square": 12,
     "Place": 13,
     "Bridge": 14
}


In [ ]:
# address_encoding = {"Street": "Street",
#  "St": "Street",
#  "Avenue": "Avenue",
#  "Ave": "Avenue",
#  "Boulevard": "Boulevard",
#  "Road": "Road",
#  "Drive": "Drive",
#  "Lane": "Lane",
#  "Tunnel":"Tunnel",
#  "Highway": "Highway",
#  "Way":"Way",
#  "Parkway":"Parkway",
#  "Parking": "Parking",
#  "Oval": "Oval",
#  "Square": "Square",
#  "Place": "Place",
#  "Bridge": "Bridge"} 

address_encoding = {
    "Street": 0,
     "St": 0,
     "Avenue": 1,
     "Ave": 1,
     "Boulevard": 2,
     "Road": 3,
     "Drive": 4,
     "Lane": 5,
     "Tunnel": 6,
     "Highway": 7,
     "Way": 8,
     "Parkway": 9,
     "Parking": 10,
     "Oval": 11,
     "Square": 12,
     "Place": 13,
     "Bridge": 14
}

In [ ]:
def encode(x):
    if pd.isna(x):
        return 15
    for road in address_encoding:
        if (road in x):
            return address_encoding[road]
    return 15

In [ ]:
train["EntryAdressEncoded"] = train['EntryStreetName'].apply(encode)
train['ExitAddressEncoded'] = train['ExitStreetName'].apply(encode)
test["EntryAdressEncoded"] = test['EntryStreetName'].apply(encode)
test['ExitAddressEncoded'] = test['ExitStreetName'].apply(encode)

In [ ]:
pd.set_option('display.max_rows', 200)
train

In [ ]:
def check_if_entry_equals_exit_address(entry_address, exit_address):
    if (entry_address == exit_address):
        return True
    return False


In [ ]:
train['IfEntryExitSame'] = train.apply(lambda x: check_if_entry_equals_exit_address(x.EntryStreetName, x.ExitStreetName), axis=1)
test['IfEntryExitSame'] = test.apply(lambda x: check_if_entry_equals_exit_address(x.EntryStreetName, x.ExitStreetName), axis=1)

In [ ]:
directions = {
    'N': 0,
    'NE': 1/4,
    'E': 1/2,
    'SE': 3/4,
    'S': 1,
    'SW': 5/4,
    'W': 3/2,
    'NW': 7/4
}

In [ ]:
train['EntryHeading'] = train['EntryHeading'].map(directions)
train['ExitHeading'] = train['ExitHeading'].map(directions)

test['EntryHeading'] = test['EntryHeading'].map(directions)
test['ExitHeading'] = test['ExitHeading'].map(directions)

In [ ]:
train['diffHeading'] = train['EntryHeading']-train['ExitHeading'] 
test['diffHeading'] = test['EntryHeading']-test['ExitHeading']

train[['ExitHeading','EntryHeading','diffHeading']].drop_duplicates().head(10)
train["same_street_exact"] = (train["EntryStreetName"] ==  train["ExitStreetName"]).astype(int)
test["same_street_exact"] = (test["EntryStreetName"] ==  test["ExitStreetName"]).astype(int)

**Intersection**

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
train["Intersection"] = train["IntersectionId"].astype(str) + train["City"]
test["Intersection"] = test["IntersectionId"].astype(str) + test["City"]

print(train["Intersection"].sample(6).values)
pd.concat([train["Intersection"],test["Intersection"]],axis=0).drop_duplicates().values

In [ ]:
le.fit(pd.concat([train["Intersection"],test["Intersection"]]).drop_duplicates().values)
train["Intersection"] = le.transform(train["Intersection"])
test["Intersection"] = le.transform(test["Intersection"])

One-hot encoding for City

In [ ]:
one_hot = pd.get_dummies(train['City'])
train = train.drop('City',axis = 1)
train = train.join(one_hot)


In [ ]:
one_hot = pd.get_dummies(test['City'])
test = test.drop('City',axis = 1)
test = test.join(one_hot)

Is Left

In [ ]:
def isLeft(entry, exit):
    if entry == exit:
        return False
    left_dir = []
    current_dir = entry
    for i in range(3):
        current_dir -= 1/4
        if current_dir < 0:
            current_dir += 2.0
        left_dir.append(current_dir)
    return exit in left_dir

In [ ]:
train['TurnLeft'] = train.apply(lambda x: isLeft(x.EntryHeading, x.ExitHeading), axis = 1)
test['TurnLeft'] = test.apply(lambda x: isLeft(x.EntryHeading, x.ExitHeading), axis = 1)

In [ ]:
train = train.dropna()

In [ ]:
train

In [ ]:
test

In [ ]:
train.columns

In [ ]:
Y = train[["TotalTimeStopped_p20", "TotalTimeStopped_p50", "TotalTimeStopped_p80", "DistanceToFirstStop_p20","DistanceToFirstStop_p50", "DistanceToFirstStop_p80" ]]
cols = [ 'IntersectionId', "Intersection", 'Latitude', 'Longitude',
         'EntryHeading', 'ExitHeading', 'Hour', 'Weekend',
       'Month', 'EntryAdressEncoded', 'ExitAddressEncoded', 'IfEntryExitSame',
       'Atlanta', 'Boston', 'Chicago', 'Philadelphia']
X = train[[ 'IntersectionId', "Intersection", 'Latitude', 'Longitude',
         'EntryHeading', 'ExitHeading', 'Hour', 'Weekend',
       'Month', 'EntryAdressEncoded', 'ExitAddressEncoded', 'IfEntryExitSame',
       'Atlanta', 'Boston', 'Chicago', 'Philadelphia']]
testX = test[cols]

# X = train 
# X = X.drop(['TotalTimeStopped_p20', 'TotalTimeStopped_p40',
#        'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
#        'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
#        'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
#        'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
#        'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
#        'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80' ], axis = 1)

In [ ]:
Y

In [ ]:
X

In [ ]:
y1 = train["TotalTimeStopped_p20"]
y2 = train["TotalTimeStopped_p50"]
y3 = train["TotalTimeStopped_p80"]
y4 = train["DistanceToFirstStop_p20"]
y5 = train["DistanceToFirstStop_p50"]
y6 = train["DistanceToFirstStop_p80"]

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [ ]:
print(X_train.shape)
print(X_validate.shape)
print(y_train.shape)
print(y_validate.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=6, random_state = 0)
model.fit(X_train, y_train)
y_pred = model.predict(X_validate)

In [ ]:
lr = RandomForestRegressor(n_estimators=6, random_state = 0)
lr.fit(X,y1)
pred1 = lr.predict(testX)
lr.fit(X,y2)
pred2 = lr.predict(testX)
lr.fit(X,y3)
pred3 = lr.predict(testX)
lr.fit(X,y4)
pred4 = lr.predict(testX)
lr.fit(X,y5)
pred5 = lr.predict(testX)
lr.fit(X,y6)
pred6 = lr.predict(testX)

In [ ]:
preds = []
for i in range(len(pred1)):
    for j in [pred1,pred2,pred3,pred4,pred5,pred6]:
        preds.append(j[i])   

lr.fit(X,Y)
predictions = lr.predict(testX)
_,C=predictions.shape

In [ ]:
with open('kaggle_submission.csv', 'w', newline='') as file:
    file.write("TargetId,Target\n")
    writer = csv.writer(file)
    R = 1920335
    for r in range(R):
        for c in range(C):
            writer.writerow([f"{r}_{c}", predictions[r][c]])

In [ ]:
#mean_squared_error(y_validate, y_pred)

In [ ]:
#model.score(X_validate, y_validate)

In [ ]:
#from sklearn import tree
#clf = tree.DecisionTreeRegressor()

In [ ]:
#clf = clf.fit(X_train,y_train)
#y_pred = clf.predict(X_validate)

In [ ]:
#mean_squared_error(y_validate, y_pred)

In [ ]:
#clf.score(X_validate, y_validate)

# cnn

In [ ]:
#from keras.models import Sequential
#from keras.layers import Dense, Conv1D, Flatten
#model = Sequential()
#model.add(Conv1D(32, 2, activation="relu", input_shape=(13,1)))
#model.add(Flatten())
#model.add(Dense(64, activation="relu"))
#model.add(Dense(1))
#model.compile(loss="mse", optimizer="adam")
#model.summary()
#estimator = KerasRegressor(model, epochs=100, batch_size=5, verbose=0)

In [ ]:
#estimator.fit(X_train, y_train)

In [ ]:
#ypred = model.predict(X_validate)
#print(model.evaluate(X_train, y_train))
#print("MSE: %.4f" % mean_squared_error(y_validate, ypred))